In [10]:
# Load env variables and create client
from dotenv import load_dotenv
from anthropic import Anthropic

load_dotenv()

client = Anthropic()
model = "claude-haiku-4-5"

In [11]:
# Helper functions
def add_user_message(messages, text):
    user_message = {"role": "user", "content": text}
    messages.append(user_message)


def add_assistant_message(messages, text):
    assistant_message = {"role": "assistant", "content": text}
    messages.append(assistant_message)


def chat(messages, system=None, temperature=0.5, stop_sequences=[]):
    params = {
        "model": model,
        "max_tokens": 1000,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences,
    }

    if system:
        params["system"] = system

    message = client.messages.create(**params)
    return message.content[0].text

In [ ]:
import json


def generate_dataset():
    prompt = """
Generate a evaluation dataset for a prompt evaluation. The dataset will be used to evaluate prompts
that generate Python, JSON, or Regex specifically for AWS-related tasks. Generate an array of JSON objects,
each representing task that requires Python, JSON, or a Regex to complete.

Example output:
```json
[
    {
        "task": "Description of task",
        "format": "python|json|regex",
        "solution_criteria": "Key criteria for evaluating the solution"
    },
    ...additional
]
```

* Focus on tasks that can be solved by writing a single Python function, a single JSON object, or a regular expression.
* Focus on tasks that do not require writing much code

Please generate 3 objects.
"""

    messages = []
    add_user_message(messages, prompt)
    add_assistant_message(messages, "```json")
    return chat(messages, stop_sequences=["```"])

In [20]:
dataset = generate_dataset()

with open("eval_dataset.json", "w") as f:
    json.dump(json.loads(dataset), f, indent=2)

In [21]:
def grade_by_model(test_case, output):
    eval_prompt = f"""
You are an expert AWS code reviewer. Your task is to evaluate the following AI-generated solution.

Original Task:
<task>
{test_case["task"]}
</task>

Solution to Evaluate:
<solution>
{output}
</solution>

Criteria for a Correct Solution:
<criteria>
{test_case["solution_criteria"]}
</criteria>

Output Format
Provide your evaluation as a structured JSON object with the following fields, in this specific order:
- "strengths": An array of 1-3 key strengths
- "weaknesses": An array of 1-3 key areas for improvement
- "reasoning": A concise explanation of your overall assessment
- "score": A number between 1-10

Respond with JSON. Keep your response concise and direct.
Example response shape:
{{
    "strengths": string[],
    "weaknesses": string[],
    "reasoning": string,
    "score": number
}}
    """

    messages = []
    add_user_message(messages, eval_prompt)
    add_assistant_message(messages, "```json")
    eval_text = chat(messages, stop_sequences=["```"])
    return json.loads(eval_text)

In [15]:
# Functions to validate the output structure
import ast
import re

def validate_json(text):
    try:
        json.loads(text.strip())
        return 10
    except json.JSONDecodeError:
        return 0


def validate_python(text):
    try:
        ast.parse(text.strip())
        return 10
    except SyntaxError:
        return 0


def validate_regex(text):
    try:
        re.compile(text.strip())
        return 10
    except re.error:
        return 0

def grade_syntax(test_case, response):
    format = test_case["format"]
    if format == "json":
        return validate_json(response)
    elif format == "python":
        return validate_python(response)
    elif format == "regex":
        return validate_regex(response)
    else:
        return 0

In [16]:
from statistics import mean


def run_prompt(test_case):
    """Merges the prompt and test case input, then returns the result"""
    prompt = f"""
    Please solve the following task:
    {test_case['task']}
    """
    messages = []
    add_user_message(messages, prompt)
    add_assistant_message(messages, "```code")
    return chat(messages, stop_sequences=["```"])

def run_test_case(test_case):
    """Calls run_prompt, then grade the result"""
    output = run_prompt(test_case)

    model_grade = grade_by_model(test_case, output)
    model_score = model_grade["score"]
    model_reasoning = model_grade["reasoning"]
    
    syntax_score = grade_syntax(test_case, output)

    return {
        "output": output,
        "test_case": test_case,
        "score": mean([model_score, syntax_score]),
        "reasoning": model_reasoning,
    }

def run_eval(dataset):
    """Loads the dataset and calls run_test_case with each test case"""
    results = []
    for test_case in dataset:
        result = run_test_case(test_case)
        results.append(result)

    average_score = mean([result["score"] for result in results])
    print(f"Average Score: {average_score}")
    
    return results

In [22]:
with open("eval_dataset.json", "r") as f:
    dataset = json.load(f)

results = run_eval(dataset)

Average Score: 6.833333333333333


In [23]:
print(json.dumps(results, indent=2))

[
  {
    "output": "\nfunction extractAWSRegion(s3Uri) {\n  // Handle s3://bucket-name/key format\n  const s3Match = s3Uri.match(/^s3:\\/\\/([a-z0-9.-]+)\\//);\n  if (s3Match) {\n    const bucketPart = s3Match[1];\n    \n    // Check if it contains region information (bucket.region.amazonaws.com)\n    const regionMatch = bucketPart.match(/\\.([a-z0-9-]+)\\.amazonaws\\.com$/);\n    if (regionMatch) {\n      return regionMatch[1];\n    }\n    \n    // No region specified, return default\n    return 'us-east-1';\n  }\n  \n  // Handle s3://bucket-name.region.amazonaws.com/key format\n  const httpsMatch = s3Uri.match(/^s3:\\/\\/([a-z0-9.-]+\\.amazonaws\\.com)\\//);\n  if (httpsMatch) {\n    const bucketPart = httpsMatch[1];\n    const regionMatch = bucketPart.match(/\\.([a-z0-9-]+)\\.amazonaws\\.com$/);\n    if (regionMatch) {\n      return regionMatch[1];\n    }\n  }\n  \n  // Default region if no match\n  return 'us-east-1';\n}\n\n// Test cases\nconsole.log(extractAWSRegion('s3://my-buck